In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import os

In [4]:
# https://chromedriver.chromium.org/downloads

driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.implicitly_wait(3)
driver.get("https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1")
seasonselem = driver.find_element_by_xpath("//select[@name='Season']")
seasons = [i.text for i in seasonselem.find_elements_by_tag_name('option')]

for season in seasons:

    # driver.get("https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season={}&SeasonType=Regular%20Season".format(season))
    driver.get("https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season={}&SeasonType=Playoffs".format(season))
    driver.implicitly_wait(2)
    try:
        statstable = driver.find_element_by_tag_name("table")
    except Exception as e:
        continue

    soup = BeautifulSoup(statstable.get_attribute("innerHTML"), features="lxml")

    columnnames = [i.text for i in soup.find_all('th') if i.text != '' and "hidden" not in i.attrs.keys()]
    columnnames.insert(0, "RANK")
    stats = []
    for tr in soup.find_all('tr'):
        td = tr.find_all('td')
        row = [i.text.strip() for i in td]
        if row != []:
            stats.append(row)

    df = pd.DataFrame(stats, columns=columnnames)
    # df.to_csv("stats_csv/teamgeneral/reg_season_stats/{}.csv".format(season))
    df.to_csv("nba_stats/{}.csv".format(season))
    # print(df)

driver.quit()

AttributeError: 'NoneType' object has no attribute 'get_attribute'